In [1]:
import os
import sys
import importlib
print(sys.path)
%pwd

os.chdir('/home/rfpred')
sys.path.append('/home/rfpred/')
sys.path.append('/home/rfpred/envs/rfenv/lib/python3.11/site-packages/')
sys.path.append('/home/rfpred/envs/rfenv/lib/python3.11/site-packages/nsdcode')


['/home/rfpred/notebooks/construction_work', '/home/rfpred/envs/rfenv/lib/python311.zip', '/home/rfpred/envs/rfenv/lib/python3.11', '/home/rfpred/envs/rfenv/lib/python3.11/lib-dynload', '', '/home/rfpred/envs/rfenv/lib/python3.11/site-packages', '/home/rfpred/notebooks/alien_nbs/lgnpy', '/home/rfpred/envs/rfenv/lib/python3.11/site-packages/nsd_access-0.0.1.dev0-py3.11.egg', '/home/rfpred/envs/rfenv/lib/python3.11/site-packages/cifti-1.1-py3.11.egg']


In [6]:

# !{sys.executable} -m pip install NeuroTools
# !{sys.executable} -m pip install SLIP
!{sys.executable} -m pip install loggabor

DEPRECATION: Loading egg at /home/rfpred/envs/rfenv/lib/python3.11/site-packages/cifti-1.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/rfpred/envs/rfenv/lib/python3.11/site-packages/nsd_access-0.0.1.dev0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached LogGabor-20240421.tar.gz (9.1 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for loggabor: filename=LogGabor-20240421-py3-none-any.whl size=8248 sha256=4dd59d01be91eb5e757e909315cd0550e156620f02bf8fe7bb2b9e82e86126a4
  Stored in directory: /home/wscheurer/.cache/pip/wheels/c8/f4/c4/a08c10a9c5127d9e111c93912b0f880c66a5db7ce0fbd02bac
Successfully built loggabor
